In [451]:
import pandas as pd
import numpy as np
import json

In [452]:
dt = pd.read_csv('trading_record.csv',index_col=False)
try:
    del dt['Date']
except:
    pass

In [453]:
dt_group = dt.groupby(['Underlying','Contract Month','Strike Price','Long/Short','Call/Put'])

labels = ['Underlying','Contract Month','Strike Price','Long/Short','Call/Put','Avg Price','Total Size']
dt_summary = pd.DataFrame([],columns=labels)
for group_name in list(dt_group.groups):
    option_data = dt_group.get_group(group_name)
    total_size = sum(option_data['Size'])
    avg_price = (sum(option_data['Price'] * option_data['Size']))/total_size
    summary = [group_name[0], group_name[1], group_name[2], group_name[3], group_name[4], avg_price, total_size]
    dt_summary = dt_summary.append(pd.Series(summary, index=labels),ignore_index = True)

def long_short(row):
    if row['Long/Short'] == 'L':
        return row['Total Size']
    if row['Long/Short'] == 'S':
        return row['Total Size'] * -1
    

dt_summary['direction'] = dt_summary[['Long/Short','Total Size']].apply(long_short, axis=1)

In [454]:
labels = ['Underlying','Contract Month','Strike Price','Call/Put','Avg Long Price','Avg Short Price','Total Size','P/L','Net Size']

dt_summary_group = dt_summary.groupby(['Underlying','Contract Month','Strike Price','Call/Put'])

option_summary = pd.DataFrame([],columns=labels)

with open('stock_lots.json') as outputfile:
    lot_size = json.load(outputfile)
    


for group_name in list(dt_summary_group.groups):
    group_data = dt_summary_group.get_group(group_name)
    
    try:
        avg_long_price = float(group_data[group_data['Long/Short'] == 'L' ]['Avg Price'])
    except:
        avg_long_price = 0
        
    try:
        avg_short_price = float(group_data[group_data['Long/Short'] == 'S' ]['Avg Price'])
    except:
        avg_short_price = 0
    
    try:
        avg_long_price = float(group_data[group_data['Long/Short'] == 'L' ]['Avg Price'])
    except:
        avg_long_price = 0
        
    try:
        avg_short_price = float(group_data[group_data['Long/Short'] == 'S' ]['Avg Price'])
    except:
        avg_short_price = 0
    
    total_size = max(group_data['Total Size'])
    
    profit_per = (avg_short_price - avg_long_price) * total_size
    
    profit = int(profit_per * int(lot_size[str(group_name[0])]))
    
    data = [group_name[0], group_name[1], group_name[2] ,group_name[3] ,avg_long_price ,avg_short_price , total_size, profit, sum(group_data['direction'])]
    
    option_summary = option_summary.append(pd.Series(data, index=labels),ignore_index = True)

In [456]:
option_summary

,Underlying,Contract Month,Strike Price,Call/Put,Avg Long Price,Avg Short Price,Total Size,P/L,Net Size
0,700,8,340.0,P,10.00000,3.644444,45,-28600,0
1,700,8,310.0,P,0.00000,2.990000,10,2990,0
2,2318,9,75.0,P,1.47000,0.000000,3,-2205,3
3,700,9,350.0,C,0.00000,1.470000,5,735,-5
4,66,8,41.0,P,0.02000,1.290000,3,1905,0
5,175,9,14.5,P,0.27000,0.000000,1,-270,1
6,66,9,40.0,C,0.84000,0.000000,3,-1260,3
7,1211,9,47.0,P,1.38000,0.000000,3,-2070,3
8,700,9,350.0,P,23.41875,10.547500,8,-10297,0
9,700,8,330.0,P,1.85320,2.350000,10,496,0


In [461]:
option_summary.sort_values(['Contract Month'], ascending=True)

,Underlying,Contract Month,Strike Price,Call/Put,Avg Long Price,Avg Short Price,Total Size,P/L,Net Size
0,700,8,340.0,P,10.00000,3.644444,45,-28600,0
1,700,8,310.0,P,0.00000,2.990000,10,2990,0
4,66,8,41.0,P,0.02000,1.290000,3,1905,0
9,700,8,330.0,P,1.85320,2.350000,10,496,0
2,2318,9,75.0,P,1.47000,0.000000,3,-2205,3
3,700,9,350.0,C,0.00000,1.470000,5,735,-5
5,175,9,14.5,P,0.27000,0.000000,1,-270,1
6,66,9,40.0,C,0.84000,0.000000,3,-1260,3
7,1211,9,47.0,P,1.38000,0.000000,3,-2070,3
8,700,9,350.0,P,23.41875,10.547500,8,-10297,0
